# Capitalize R&D expenses
This notebook calculates the value of research asset and current year's amortization cost.

Guideline for choosing the amortization period:
- Non-technological service: 2 years
- Retail, tech service: 3 years
- Light manufacturing: 5 years
- Heavy manufacturing: 10 years
- Research, with patenting: 10 years
- Long gestation period: 10 years

In [1]:
# Define the total period over which R&D costs are amortized
amortization_period_years = 5

# R&D Costs for the current year (Year 0) and preceding years (Year -1, -2, ...)
# The list should contain costs for 'amortization_period_years' number of years.
# Index 0 = Current Year (Year 0)
# Index 1 = Prior Year (Year -1)
# Index 2 = Year before prior (Year -2)
# ... up to index (amortization_period_years - 1)
# Example: If period is 5 years, we need costs for years 0, -1, -2, -3, -4.
# Ensure the list has exactly 'amortization_period_years' elements.
# Example costs (replace with actuals):
#                  Year 0,   Year -1, Year -2, Year -3, Year -4
rd_costs_by_year = [100000,   90000,   80000,   70000,   60000]


In [2]:
import pandas as pd 


def calculate_research_asset_amortization():
    print(f"Amortization Period: {amortization_period_years} years")
    print(f"Input R&D Costs (Current to Oldest): {rd_costs_by_year}\n")

    # --- Input Validation ---
    if amortization_period_years <= 0:
        print("Error: Amortization period must be greater than zero.")
        return
    if not isinstance(rd_costs_by_year, list):
         print("Error: R&D costs must be provided as a list.")
         return
    if len(rd_costs_by_year) != amortization_period_years:
        print(f"Error: Expected {amortization_period_years} R&D cost entries, but got {len(rd_costs_by_year)}.")
        print(f"       Provide costs for the current year and the preceding {amortization_period_years - 1} years.")
        return
    if any(cost < 0 for cost in rd_costs_by_year):
        print("Error: R&D costs cannot be negative.")
        return

    # --- Calculation ---
    calculation_data = []
    total_unamortized_value = 0.0
    total_current_amortization = 0.0

    for year_index, rd_cost in enumerate(rd_costs_by_year):
        # year_index 0 = current year, 1 = prior year, etc.

        # Let year 1 be the *first* year of amortization (the year cost is incurred).
        # Cost from current year (index 0) is in its 1st year.
        # Cost from last year (index 1) is in its 2nd year.
        # Cost from index `i` is in its `i+1` year.
        years_amortized = year_index + 1

        # Calculate the unamortized ratio *at the end of the current year*
        # Ratio = (Total Years - Years Amortized) / Total Years
        unamortized_ratio = (amortization_period_years - years_amortized) / amortization_period_years
        # Ensure ratio isn't negative if cost is fully amortized
        unamortized_ratio = max(0.0, unamortized_ratio) # Ensure float division and max with 0.0

        # Calculate the portion of this year's cost that remains unamortized *at the end of the current year*
        unamortized_portion = rd_cost * unamortized_ratio

        # Calculate the portion of this year's cost that is amortized *in the current period*
        # Straight-line expense this year = Original Cost / Total Years
        current_year_amortization_for_this_cost = rd_cost / amortization_period_years

        # Add to totals
        total_unamortized_value += unamortized_portion
        total_current_amortization += current_year_amortization_for_this_cost

        calculation_data.append({
            "Year Incurred Index": f"Year -{year_index}" if year_index > 0 else "Current Year (0)",
            "R&D Cost": rd_cost,
            "Years Amortized": years_amortized,
            "Unamortized Ratio": unamortized_ratio, # Added ratio here
            "Unamortized Portion End of Year": unamortized_portion,
            "Amortization Expense This Year": current_year_amortization_for_this_cost
        })

    # --- Display Results ---
    print("\n--- Detailed Calculation ---")
    df = pd.DataFrame(calculation_data)

    # Define desired column order
    column_order = [
        "Year Incurred Index",
        "R&D Cost",
        "Years Amortized",
        "Unamortized Ratio",
        "Unamortized Portion End of Year",
        "Amortization Expense This Year"
    ]
    df = df[column_order] # Reorder columns

    # Format columns for currency and percentage
    pd.options.display.float_format = '${:,.2f}'.format # Default format for floats
    df['R&D Cost'] = df['R&D Cost'].map('${:,.2f}'.format)
    df['Unamortized Ratio'] = df['Unamortized Ratio'].map('{:.2%}'.format) # Format as percentage
    df['Unamortized Portion End of Year'] = df['Unamortized Portion End of Year'].map('${:,.2f}'.format)
    df['Amortization Expense This Year'] = df['Amortization Expense This Year'].map('${:,.2f}'.format)

    # Print DataFrame without index and adjust display width if needed
    # You might need to configure pandas display options for wider tables if it wraps
    # pd.set_option('display.width', 1000) # Example if needed
    print(df.to_string(index=False))

    print("\n" + "-" * 80)
    print("## Summary Totals")
    print("-" * 80)
    print(f"Total Research Asset Value (End of Current Year): ${total_unamortized_value:,.2f}")
    print(f"Total Amortization Expense (Current Year):       ${total_current_amortization:,.2f}")
    print("-" * 80) 

# --- Main execution ---
if __name__ == "__main__":
    try:
        calculate_research_asset_amortization()
    except Exception as e:
        print(f"\nAn error occurred: {e}")



Amortization Period: 5 years
Input R&D Costs (Current to Oldest): [100000, 90000, 80000, 70000, 60000]


--- Detailed Calculation ---
Year Incurred Index    R&D Cost  Years Amortized Unamortized Ratio Unamortized Portion End of Year Amortization Expense This Year
   Current Year (0) $100,000.00                1            80.00%                      $80,000.00                     $20,000.00
            Year -1  $90,000.00                2            60.00%                      $54,000.00                     $18,000.00
            Year -2  $80,000.00                3            40.00%                      $32,000.00                     $16,000.00
            Year -3  $70,000.00                4            20.00%                      $14,000.00                     $14,000.00
            Year -4  $60,000.00                5             0.00%                           $0.00                     $12,000.00

--------------------------------------------------------------------------------
## S